<a href="https://colab.research.google.com/github/TacosyHorchata/autoprintify/blob/main/Printify_bulk_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

github repo: https://github.com/TacosyHorchata/autoprintify

## HEY!

This program allows you to bulk upload your ready designs into your Printify store.

Please keep in mind:

- Images must be in the /images folder.
- Images must be in .png format if you want to maintain a transparent background.
- The filenames of the images should follow the format: title + 'Product'.
- T-shirts and mugs will be created for each image in the /images folder.

Follow the instructions below (approximately 5 minutes):

## 1. Create the images folder and import the necessary packages

Simply run the cell.

In [2]:
import requests
import base64
import os

# Create the images directory if it does not exist
if not os.path.exists('images'):
    os.mkdir('images')


## 2. Move the desired images into the /images folder.

## 3. API Key - Enter your API key  
Navigate to Printify > Account > Connections > API Tokens.

In [18]:
# Define the API key (please ensure to replace with your actual key, you can find it in Printify > Account > Connections > API tokens)
api_key = ''

## 4. Get Your Shop ID

The Shop ID is unique for each store—Etsy, Shopify, Printify, etc.

#### Copy and paste the desired Shop ID in the third step.

In [ ]:
import requests

def get_shop_ids():
    """Fetch and display the available shop IDs from the Printify API."""

    response = requests.get('https://api.printify.com/v1/shops.json', headers={
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json',
    })

    shops = []
    if response.status_code in [200, 201]:
        shops = response.json()
    if shops:
        shop_ids = {shop['id']: shop['title'] for shop in shops}
        for shop_id, title in shop_ids.items():
            print(f"Shop ID: {shop_id}, Name: {title}")
    else:
        print("No shops found.")

print(get_shop_ids())

Here’s a refined version:

---

## 5. Edit These Variables

#### You can find the Blueprint ID in the URL of the product, for example:

`https://printify.com/app/products/145/gildan/unisex-softstyle-t-shirt`

In this case, "145" is the Blueprint ID.

#### The same applies for the Provider ID:

`https://printify.com/app/print-provider/42/drive-fulfillment`

Here, "42" is the Provider ID.

In [21]:
## Shop ID printed in 2nd step ^^
shop_id = '' #Replace with the Shop ID you fetched in the second step e.g. '12000033'

## Blueprints of Printify products
blueprint_tshirt_id = 145  # Replace with tshirt id
blueprint_mug_id = 68  # Replace with mug id
print_provider_tshirt_id = 42  # Replace with tshirt provider id
print_provider_mugs_id = 1  # Replace with mug provider id

## Color for tshirt (Just 1 supported)
color = 'White' #just for tshirt

## Retail Price
mug_price = 1757 #17.57 USD
tshirt_price = 2422 #24.22 USD

##
tshirt_title = 'T-shirt'
mug_title = 'Mug'
mug_description = 'Good product'

# Specify the directory containing your images
image_directory = 'images'  # Replace with the actual path

#### Product Descriptions - Edit as Needed

In [10]:
tshirt_description = '''
- 100% ring-spun cotton (Heather, Sport Grey, and Graphite Heather include cotton/polyester blends)
- Lightweight fabric (4.5 oz/yd² or 153 g/m²) for year-round comfort
- Classic fit with a versatile crew neckline
- Tear-away label for added comfort
- Ethically sourced US cotton, Oeko-Tex certified for safety and quality
- Sustainable production as a proud member of the US Cotton Trust Protocol

Elevate your wardrobe with this premium unisex t-shirt, crafted from 100% ring-spun cotton for unmatched softness and comfort. The lightweight fabric (4.5 oz/yd²) makes it perfect for any season, offering breathability and durability you can rely on. With a classic fit and versatile crew neckline, this tee easily transitions from casual to semi-formal settings, making it a go-to for any occasion.

Designed for comfort, each shirt features a tear-away label to eliminate any irritation. Made from ethically sourced US cotton and certified by Oeko-Tex for safety, you can feel good about wearing sustainable fashion. Heather colors, Sport Grey, and Graphite Heather include a blend of cotton and polyester for added style and flexibility.

'''

mug_description = '''
- White ceramic
- 11oz capacity (0.33 l)
- Rounded corners for a sleek design
- Comfortable C-handle for easy grip
- Lead and BPA-free for safe drinking

Start your day right with this classic 11oz white ceramic mug, perfect for your morning coffee, tea, or any beverage of choice. Designed with a sleek, minimalist look, its rounded corners and comfortable C-handle provide a cozy grip. Made from durable, lead and BPA-free materials, this mug ensures safe, long-lasting use. Whether you're at home, in the office, or gifting a loved one, this versatile mug is both practical and stylish, making it a must-have for any kitchen collection.

'''

## 5. Run all the following cells

In [11]:
# Product configurations
product_configs = [
    {
        "title": "T-shirt",
        "description": tshirt_description,
        "blueprint_id": blueprint_tshirt_id,
        "print_provider_id": print_provider_tshirt_id,
        "color": "White",
        "price": tshirt_price,
        "position_scale": 0.6,
        "position_x": 0.5,
        "position_y": 0.25
    },
    {
        "title": "Mug",
        "description": mug_description,
        "blueprint_id": blueprint_mug_id,
        "print_provider_id": print_provider_mugs_id,
        "color": "White",
        "price": mug_price,
        "position_scale": 0.3,
        "position_x": 0.5,
        "position_y": 0.5
    }
]

In [12]:
def request_printify_api(endpoint, method='GET', payload=None):
    """Reusable function to interact with the Printify API."""
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json',
    }

    url = f'https://api.printify.com/v1/{endpoint}'
    if method == 'GET':
        response = requests.get(url, headers=headers)
    else:
        response = requests.post(url, json=payload, headers=headers)

    if response.status_code in [200, 201]:
        return response.json()
    else:
        print(f'Error: {response.json()}')
        return None

In [13]:
def upload_image_by_base64(file_name, file_path):
    """Upload an image to Printify in base64 format and return the image ID."""
    try:
        with open(file_path, 'rb') as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

        payload = {
            "file_name": file_name,
            "contents": encoded_string
        }

        response = request_printify_api('uploads/images.json', method='POST', payload=payload)
        if response:
            print(f'Successfully uploaded image: {file_name}')
            return response['id']
    except Exception as e:
        print(f'Error uploading image: {e}')
    return None

In [14]:
def fetch_variants(blueprint_id, print_provider_id):
    """Retrieve available variants for a given blueprint and print provider."""
    endpoint = f'catalog/blueprints/{blueprint_id}/print_providers/{print_provider_id}/variants.json'
    return request_printify_api(endpoint)

In [15]:
def create_product(title, description, blueprint_id, image_id, variants, print_provider_id, price, shop_id, position_scale, position_x, position_y):
    """Create a product (generalized for both t-shirts and mugs)."""
    for variant in variants:
        variant['price'] = price

    product_data = {
        "title": title,
        "description": description,
        "blueprint_id": blueprint_id,
        "print_provider_id": print_provider_id,
        "variants": variants,
        "print_areas": [
            {
                "variant_ids": [v['id'] for v in variants],
                "placeholders": [
                    {
                        "position": "front",
                        "images": [
                            {
                                "id": image_id,
                                "x": position_x,
                                "y": position_y,
                                "scale": position_scale,
                                "angle": 0
                            },
                        ],
                    },
                ],
            },
        ],
    }

    response = request_printify_api(f'shops/{shop_id}/products.json', method='POST', payload=product_data)
    if response:
        print(f'Created product: {response}')
    else:
        print(f'Error creating product.')

In [16]:
def create_products_for_images(image_directory, product_configs, shop_id):
    """Create products for each image in the directory based on product configurations."""
    for image_file in os.listdir(image_directory):
        if image_file.endswith(('.png', '.jpg', '.jpeg', '.gif')):
            file_path = os.path.join(image_directory, image_file)
            image_id = upload_image_by_base64(image_file, file_path)

            if image_id:
                for config in product_configs:
                    print(config, 'CONFIG')
                    product = fetch_variants(config['blueprint_id'], config['print_provider_id'])
                    if config['title'] == 'T-shirt':
                        filtered_variants = [v for v in product['variants'] if v.get('options', {}).get('color') == config['color']]
                    elif config['title'] == 'Mug':
                        filtered_variants = [v for v in product['variants']]
                    #filtered_variants = [v for v in product['variants'] if v.get('options', {}).get('color') == config['color']]
                    print(filtered_variants)
                    if filtered_variants:
                        title = f'{image_file.split(".")[0]} {config["title"]}'.title()
                        create_product(
                            title=title,
                            description=config['description'],
                            blueprint_id=config['blueprint_id'],
                            image_id=image_id,
                            variants=filtered_variants,
                            print_provider_id=config['print_provider_id'],
                            price=config['price'],
                            shop_id=shop_id,
                            position_scale=config['position_scale'],
                            position_x=config['position_x'],
                            position_y=config['position_y']
                        )

In [ ]:
# Call the function to create products
create_products_for_images('images', product_configs, shop_id)